# Load necessary packages

In [1]:
import pandas as pd
import numpy as np

hex_salmon = '#F68F83'
hex_gold = '#BC9661'
hex_indigo = '#2D2E5F'
hex_maroon = '#8C4750'
hex_white = '#FAFAFA'
hex_blue = '#7EB5D2'

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.dates import DateFormatter
import matplotlib.dates as dates

import matplotlib.font_manager as font_manager
mpl.font_manager._rebuild()

mpl.rcParams['font.family'] = 'SF Mono'
mpl.rcParams['font.weight'] = 'medium'
mpl.rcParams['axes.titleweight'] = 'semibold'
mpl.rcParams['axes.labelweight'] = 'medium'
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[hex_indigo, hex_salmon, hex_maroon])
mpl.rcParams["figure.titlesize"] = 'large'
mpl.rcParams["figure.titleweight"] = 'semibold'

from termcolor import colored

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, accuracy_score

import tensorflow as tf

# Organise data

## Import features

In [2]:
! pip install 'git+git://github.com/HR/github-clone#egg=ghclone' &> /dev/null

! ghclone https://github.com/timovijn/ElectricityPriceForecasting/tree/master/LSTM

zsh:1: command not found: ghclone


In [3]:
features = pd.read_pickle(f"./features.pkl")

display(features)

,ID3,VOL,MCP,LOAD,LOAD_F,LOAD_FE,ID3 (-4),ID3 (-5),ID3 (-6),ID3 (-7),...,HOD 14,HOD 15,HOD 16,HOD 17,HOD 18,HOD 19,HOD 20,HOD 21,HOD 22,HOD 23
2015-01-08 01:00:00+00:00,22.953776,439.5,32.32,9008.00,8505.25,502.75,29.934792,61.666667,61.118812,61.370370,...,0,0,0,0,0,0,0,0,0,0
2015-01-08 02:00:00+00:00,23.168355,261.5,31.10,8889.25,8222.25,667.00,29.853669,29.934792,61.666667,61.118812,...,0,0,0,0,0,0,0,0,0,0
2015-01-08 03:00:00+00:00,21.000000,420.5,30.17,8929.25,8122.25,807.00,24.012378,29.853669,29.934792,61.666667,...,0,0,0,0,0,0,0,0,0,0
2015-01-08 04:00:00+00:00,30.000000,460.6,24.54,9423.75,8323.50,1100.25,23.269810,24.012378,29.853669,29.934792,...,0,0,0,0,0,0,0,0,0,0
2015-01-08 05:00:00+00:00,30.000000,250.0,32.00,10884.50,9015.00,1869.50,22.953776,23.269810,24.012378,29.853669,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-30 14:00:00+00:00,53.790740,446.6,46.19,13842.50,15329.25,1486.75,76.370821,87.755884,78.709213,52.958116,...,1,0,0,0,0,0,0,0,0,0
2018-12-30 15:00:00+00:00,59.477646,131.6,47.64,14319.25,15644.50,1325.25,63.690401,76.370821,87.755884,78.709213,...,0,1,0,0,0,0,0,0,0,0
2018-12-30 16:00:00+00:00,59.883829,310.1,55.94,15120.75,16285.75,1165.00,56.170316,63.690401,76.370821,87.755884,...,0,0,1,0,0,0,0,0,0,0
2018-12-30 17:00:00+00:00,59.471501,220.9,58.40,14728.75,15555.75,827.00,51.675229,56.170316,63.690401,76.370821,...,0,0,0,1,0,0,0,0,0,0


## Select features

In [4]:
X = features[['ID3', 'LOAD']]
y = features[['ID3']]

lag_X = range(-4, -7, -1)
lag_y = [0]

X2 = pd.DataFrame(index = X.index, columns = pd.MultiIndex.from_product([['X'], X.columns, lag_X], names = ['Type', 'Feature', 'Lag']))
X2 = X2.rename_axis('Timestamp')

y2 = pd.DataFrame(index = y.index, columns = pd.MultiIndex.from_product([['y'], y.columns, lag_y], names = ['Type', 'Feature', 'Lag']))
y2 = y2.rename_axis('Timestamp')

frame = pd.merge(y2, X2, left_index = True, right_index = True)

X3 = pd.DataFrame(index = X.index)
y3 = pd.DataFrame(index = y.index)

for c in X.columns:
    for l in lag_X:
        X3[f'{c} ({l})'] = X[f'{c}'].shift(-l)

frame['X'] = X3.values

for c in y.columns:
    for l in lag_y:
        y3[f'{c} ({l})'] = y.shift(-l)

frame['y'] = y3.values

frame = frame.dropna()

display(frame)

Type                               y          X                        \
Feature                          ID3        ID3                         
Lag                                0         -4         -5         -6   
Timestamp                                                               
2015-01-08 07:00:00+00:00  43.588694  21.000000  23.168355  22.953776   
2015-01-08 08:00:00+00:00  43.537764  30.000000  21.000000  23.168355   
2015-01-08 09:00:00+00:00  48.252186  30.000000  30.000000  21.000000   
2015-01-08 10:00:00+00:00  48.683607  43.153846  30.000000  30.000000   
2015-01-08 11:00:00+00:00  46.580903  43.588694  43.153846  30.000000   
...                              ...        ...        ...        ...   
2018-12-30 14:00:00+00:00  53.790740  76.370821  87.755884  78.709213   
2018-12-30 15:00:00+00:00  59.477646  63.690401  76.370821  87.755884   
2018-12-30 16:00:00+00:00  59.883829  56.170316  63.690401  76.370821   
2018-12-30 17:00:00+00:00  59.471501  51.675229  56.170316  63.690401   
2018-12-30 18:00:00+00:00  58.448950  53.790740  51.675229  56.170316   

Type                                                     
Feature                        LOAD                      
Lag                              -4        -5        -6  
Timestamp                                                
2015-01-08 07:00:00+00:00   8929.25   8889.25   9008.00  
2015-01-08 08:00:00+00:00   9423.75   8929.25   8889.25  
2015-01-08 09:00:00+00:00  10884.50   9423.75   8929.25  
2015-01-08 10:00:00+00:00  13364.50  10884.50   9423.75  
2015-01-08 11:00:00+00:00  15053.25  13364.50  10884.50  
...                             ...       ...       ...  
2018-12-30 14:00:00+00:00  13448.25  13035.50  12507.50  
2018-12-30 15:00:00+00:00  13715.50  13448.25  13035.50  
2018-12-30 16:00:00+00:00  13770.00  13715.50  13448.25  
2018-12-30 17:00:00+00:00  13748.00  13770.00  13715.50  
2018-12-30 18:00:00+00:00  13842.50  13748.00  13770.00  

[33839 rows x 7 columns]

## Split train and test

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    frame['X'],
    frame['y'],
    test_size = 0.3,
    random_state = 0,
    shuffle = False)

X_train.columns = pd.MultiIndex.from_product([['X'], X.columns, lag_X], names = ['Type', 'Feature', 'Lag'])
y_train.columns = pd.MultiIndex.from_product([['y'], y.columns, lag_y], names = ['Type', 'Feature', 'Lag'])

frame_train = pd.merge(y_train, X_train, left_index = True, right_index = True)

X_test.columns = pd.MultiIndex.from_product([['X'], X.columns, lag_X], names = ['Type', 'Feature', 'Lag'])
y_test.columns = pd.MultiIndex.from_product([['y'], y.columns, lag_y], names = ['Type', 'Feature', 'Lag'])

frame_test = pd.merge(y_test, X_test, left_index = True, right_index = True)

print()
print(f'Train input', frame_train['X'].shape, 'output', frame_train['y'].shape)
print()
print(f'Test input', frame_test['X'].shape, 'output', frame_test['y'].shape)
print()

display(frame_train)


Train input (23687, 6) output (23687, 1)

Test input (10152, 6) output (10152, 1)



Type                               y          X                        \
Feature                          ID3        ID3                         
Lag                                0         -4         -5         -6   
Timestamp                                                               
2015-01-08 07:00:00+00:00  43.588694  21.000000  23.168355  22.953776   
2015-01-08 08:00:00+00:00  43.537764  30.000000  21.000000  23.168355   
2015-01-08 09:00:00+00:00  48.252186  30.000000  30.000000  21.000000   
2015-01-08 10:00:00+00:00  48.683607  43.153846  30.000000  30.000000   
2015-01-08 11:00:00+00:00  46.580903  43.588694  43.153846  30.000000   
...                              ...        ...        ...        ...   
2017-10-28 13:00:00+00:00  43.582655  35.730954  35.416084  35.416372   
2017-10-28 14:00:00+00:00  40.537582  37.779619  35.730954  35.416084   
2017-10-28 15:00:00+00:00  41.788302  43.600365  37.779619  35.730954   
2017-10-28 16:00:00+00:00  45.834303  42.941124  43.600365  37.779619   
2017-10-28 17:00:00+00:00  49.971656  43.582655  42.941124  43.600365   

Type                                                     
Feature                        LOAD                      
Lag                              -4        -5        -6  
Timestamp                                                
2015-01-08 07:00:00+00:00   8929.25   8889.25   9008.00  
2015-01-08 08:00:00+00:00   9423.75   8929.25   8889.25  
2015-01-08 09:00:00+00:00  10884.50   9423.75   8929.25  
2015-01-08 10:00:00+00:00  13364.50  10884.50   9423.75  
2015-01-08 11:00:00+00:00  15053.25  13364.50  10884.50  
...                             ...       ...       ...  
2017-10-28 13:00:00+00:00  13883.50  13757.25  13372.25  
2017-10-28 14:00:00+00:00  13822.25  13883.50  13757.25  
2017-10-28 15:00:00+00:00  13629.50  13822.25  13883.50  
2017-10-28 16:00:00+00:00  13506.00  13629.50  13822.25  
2017-10-28 17:00:00+00:00  13532.25  13506.00  13629.50  

[23687 rows x 7 columns]

## Scaling

In [6]:
frame_train_unscaled = frame_train
frame_test_unscaled = frame_test

y_scaler = StandardScaler()
y_scaler.fit(frame_train['y'])

frame_train['y'] = y_scaler.transform(frame_train['y'])
frame_test['y'] = y_scaler.transform(frame_test['y'])

X_scaler = StandardScaler()
X_scaler.fit(frame_train['X'])

frame_train['X'] = X_scaler.transform(frame_train['X'])
frame_test['X'] = X_scaler.transform(frame_test['X'])

display(frame_train)

Type                              y         X                                \
Feature                         ID3       ID3                          LOAD   
Lag                               0        -4        -5        -6        -4   
Timestamp                                                                     
2015-01-08 07:00:00+00:00  0.334279 -1.143725 -1.001758 -1.015724 -1.394870   
2015-01-08 08:00:00+00:00  0.330946 -0.554789 -1.143647 -1.001683 -1.191284   
2015-01-08 09:00:00+00:00  0.639453 -0.554789 -0.554722 -1.143569 -0.589895   
2015-01-08 10:00:00+00:00  0.667684  0.305964 -0.554722 -0.554656  0.431119   
2015-01-08 11:00:00+00:00  0.530086  0.334420  0.306015 -0.554656  1.126376   
...                             ...       ...       ...       ...       ...   
2017-10-28 13:00:00+00:00  0.333883 -0.179770 -0.200314 -0.200236  0.644791   
2017-10-28 14:00:00+00:00  0.134617 -0.045711 -0.179710 -0.200255  0.619574   
2017-10-28 15:00:00+00:00  0.216463  0.335183 -0.045654 -0.179652  0.540219   
2017-10-28 16:00:00+00:00  0.481229  0.292044  0.335233 -0.045598  0.489374   
2017-10-28 17:00:00+00:00  0.751973  0.334024  0.292095  0.335282  0.500182   

Type                                           
Feature                                        
Lag                              -5        -6  
Timestamp                                      
2015-01-08 07:00:00+00:00 -1.411205 -1.362193  
2015-01-08 08:00:00+00:00 -1.394738 -1.411079  
2015-01-08 09:00:00+00:00 -1.191160 -1.394612  
2015-01-08 10:00:00+00:00 -0.589793 -1.191041  
2015-01-08 11:00:00+00:00  0.431184 -0.589694  
...                             ...       ...  
2017-10-28 13:00:00+00:00  0.592873  0.434438  
2017-10-28 14:00:00+00:00  0.644848  0.592931  
2017-10-28 15:00:00+00:00  0.619632  0.644904  
2017-10-28 16:00:00+00:00  0.540280  0.619689  
2017-10-28 17:00:00+00:00  0.489437  0.540340  

[23687 rows x 7 columns]

In [7]:
step1 = []
step2 = []
step3 = []

for index, row in frame_train.iterrows():
    step2 = []
    for l in lag_X:
        step1 = []
        for c in X.columns:
            step1.append(row['X'][f'{c}'][l])
        step2.append(step1)
    step3.append(step2)

X_train = step3

X_train = np.array(X_train)

In [8]:
step1 = []
step2 = []
step3 = []

for index, row in frame_test.iterrows():
    step2 = []
    for l in lag_X:
        step1 = []
        for c in X.columns:
            step1.append(row['X'][f'{c}'][l])
        step2.append(step1)
    step3.append(step2)

X_test = step3

X_test = np.array(X_test)

# Learning

## Create model

In [9]:
BATCH_SIZE = 32
EPOCHS = 10

In [10]:
from keras.models import Sequential
from keras.layers import Dense, GRU
from keras.optimizers import SGD, Adam
from keras.utils.vis_utils import plot_model
from keras.layers import GRU, Dense, RepeatVector, TimeDistributed, Flatten
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, GRU, LSTM
from keras.optimizers import SGD, Adam

In [11]:
import keras
import keras.backend as K

class Dropout(keras.layers.Dropout):
  """Applies Dropout to the input.
  The Dropout layer randomly sets input units to 0 with a frequency of `rate`
  at each step during training time, which helps prevent overfitting.
  Inputs not set to 0 are scaled up by 1/(1 - rate) such that the sum over
  all inputs is unchanged.
  Note that the Dropout layer only applies when `training` is set to True
  such that no values are dropped during inference. When using `model.fit`,
  `training` will be appropriately set to True automatically, and in other
  contexts, you can set the kwarg explicitly to True when calling the layer.
  (This is in contrast to setting `trainable=False` for a Dropout layer.
  `trainable` does not affect the layer's behavior, as Dropout does
  not have any variables/weights that can be frozen during training.)
  >>> tf.random.set_seed(0)
  >>> layer = tf.keras.layers.Dropout(.2, input_shape=(2,))
  >>> data = np.arange(10).reshape(5, 2).astype(np.float32)
  >>> print(data)
  [[0. 1.]
   [2. 3.]
   [4. 5.]
   [6. 7.]
   [8. 9.]]
  >>> outputs = layer(data, training=True)
  >>> print(outputs)
  tf.Tensor(
  [[ 0.    1.25]
   [ 2.5   3.75]
   [ 5.    6.25]
   [ 7.5   8.75]
   [10.    0.  ]], shape=(5, 2), dtype=float32)
  Args:
    rate: Float between 0 and 1. Fraction of the input units to drop.
    noise_shape: 1D integer tensor representing the shape of the
      binary dropout mask that will be multiplied with the input.
      For instance, if your inputs have shape
      `(batch_size, timesteps, features)` and
      you want the dropout mask to be the same for all timesteps,
      you can use `noise_shape=(batch_size, 1, features)`.
    seed: A Python integer to use as random seed.
  Call arguments:
    inputs: Input tensor (of any rank).
    training: Python boolean indicating whether the layer should behave in
      training mode (adding dropout) or in inference mode (doing nothing).
  """
  
  def __init__(self, rate, training=None, noise_shape=None, seed=None, **kwargs):
    super(Dropout, self).__init__(rate, noise_shape=None, seed=None, **kwargs)
    self.training = training

  def call(self, inputs, training=None):
    if 0. < self.rate < 1.:
        noise_shape = self._get_noise_shape(inputs)

        def dropped_inputs():
            return K.dropout(inputs, self.rate, noise_shape, seed=self.seed)

        if not training:
            return K.in_train_phase(dropped_inputs, inputs, training=self.training)
        return K.in_train_phase(dropped_inputs, inputs, training=training)
    return inputs

In [16]:
T = 3
HORIZON = 1

model = Sequential()

model.add(LSTM(units = 200, return_sequences = True, input_shape = (T, 2), activation='tanh'))

model.add(Dropout(0.2, training=True))

model.add(LSTM(units = 100, return_sequences = False))

model.add(Dropout(0.2, training=True))

# model.add(LSTM(units = 50, return_sequences = False))

model.add(Dense(units = 1, activation = 'relu'))

optimizer = Adam(clipvalue = 0.5)

# # Compiling
# model.compile(optimizer=optimizer,loss='mean_squared_error')

# # Fitting to the training set
# model.fit(X_train,y_train,epochs=5,batch_size=32, verbose=1)

# prediction = model.predict(X_test)
# prediction = y_scaler.inverse_transform(prediction)

In [17]:
model.compile(optimizer = 'Adam', loss = 'mse')

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 3, 200)            162400    
_________________________________________________________________
dropout_4 (Dropout)          (None, 3, 200)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 282,901
Trainable params: 282,901
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(X_train,
          np.array(frame_train['y']),
          batch_size = BATCH_SIZE,
          epochs = EPOCHS,
        #   validation_data=(valid_inputs['X'], valid_inputs['target']),
        #   callbacks=[earlystop],
          verbose = 1)

Epoch 1/10
741/741 [==============================] - 11s 10ms/step - loss: 0.8117
Epoch 2/10
741/741 [==============================] - 10s 14ms/step - loss: 0.8064
Epoch 3/10
741/741 [==============================] - 19s 26ms/step - loss: 0.7495
Epoch 4/10
741/741 [==============================] - 12s 16ms/step - loss: 0.7680
Epoch 5/10
741/741 [==============================] - 12s 17ms/step - loss: 0.8508
Epoch 6/10
741/741 [==============================] - 10s 14ms/step - loss: 0.7829
Epoch 7/10
741/741 [==============================] - 8s 10ms/step - loss: 0.7543
Epoch 8/10
741/741 [==============================] - 8s 11ms/step - loss: 0.7661
Epoch 9/10
741/741 [==============================] - 13s 17ms/step - loss: 0.7828
Epoch 10/10
741/741 [==============================] - 12s 16ms/step - loss: 0.7526


In [20]:
predictions = pd.DataFrame(index = frame_test.index, columns = pd.MultiIndex.from_product([['Prediction'], y.columns, lag_y], names = ['Type', 'Feature', 'Lag']))

frame_test = pd.merge(frame_test, predictions, left_index = True, right_index = True)

frame_test['Prediction'] = model.predict(X_test)

frame_test['Prediction'] = y_scaler.inverse_transform(frame_test['Prediction'])

frame_test['y'] = y_scaler.inverse_transform(frame_test['y'])

display(frame_test)

Type                               y         X                                \
Feature                          ID3       ID3                          LOAD   
Lag                                0        -4        -5        -6        -4   
Timestamp                                                                      
2017-10-28 18:00:00+00:00  53.021902  0.134763  0.334074  0.292145  0.624926   
2017-10-28 19:00:00+00:00  17.579661  0.216607  0.134817  0.334123  0.860110   
2017-10-28 20:00:00+00:00  18.143989  0.481366  0.216659  0.134869  0.892222   
2017-10-28 21:00:00+00:00  17.683110  0.752104  0.481414  0.216710  0.720852   
2017-10-28 22:00:00+00:00  17.911371  0.951704  0.752146  0.481459  0.450983   
...                              ...       ...       ...       ...       ...   
2018-12-30 14:00:00+00:00  53.790740  2.479596  3.224600  2.632623  0.465599   
2018-12-30 15:00:00+00:00  59.477646  1.649822  2.479605  3.224591  0.575625   
2018-12-30 16:00:00+00:00  59.883829  1.157728  1.649848  2.479611  0.598063   
2018-12-30 17:00:00+00:00  59.471501  0.863581  1.157762  1.649870  0.589006   
2018-12-30 18:00:00+00:00  58.448950  1.002015  0.863621  1.157794  0.627911   

Type                                          Prediction  
Feature                                              ID3  
Lag                              -5        -6          0  
Timestamp                                                 
2017-10-28 18:00:00+00:00  0.500244  0.489499  39.547962  
2017-10-28 19:00:00+00:00  0.624984  0.500305  39.534679  
2017-10-28 20:00:00+00:00  0.860159  0.625041  41.552841  
2017-10-28 21:00:00+00:00  0.892270  0.860207  42.600204  
2017-10-28 22:00:00+00:00  0.720906  0.892318  44.422585  
...                             ...       ...        ...  
2018-12-30 14:00:00+00:00  0.295740  0.078446  70.904869  
2018-12-30 15:00:00+00:00  0.465662  0.295808  56.033882  
2018-12-30 16:00:00+00:00  0.575685  0.465725  50.981625  
2018-12-30 17:00:00+00:00  0.598122  0.575743  47.819466  
2018-12-30 18:00:00+00:00  0.589065  0.598179  50.019669  

[10152 rows x 8 columns]

In [22]:
n_experiments = 50

test_uncertainty_df = pd.DataFrame()

for i in range(1, n_experiments + 1):
  experiment_prediction = model.predict(X_test)
  test_uncertainty_df['ID3_{}'.format(i)] = np.concatenate(y_scaler.inverse_transform(experiment_prediction), axis = 0)
  
  if i % 1 == 0:
    print(f'Experiment: {i}/{n_experiments}')

# log_energy_consumption_df = test_uncertainty_df.filter(like='ID3', axis=1)
test_uncertainty_df['ID3_mean'] = test_uncertainty_df.mean(axis=1)
test_uncertainty_df['ID3_std'] = test_uncertainty_df.std(axis=1)

test_uncertainty_df = test_uncertainty_df[['ID3_mean', 'ID3_std']]

test_uncertainty_df['lower_bound'] = test_uncertainty_df['ID3_mean'] - 3*test_uncertainty_df['ID3_std']
test_uncertainty_df['upper_bound'] = test_uncertainty_df['ID3_mean'] + 3*test_uncertainty_df['ID3_std']

print(), print(test_uncertainty_df);

Experiment: 1/50
Experiment: 2/50
Experiment: 3/50
Experiment: 4/50
Experiment: 5/50
Experiment: 6/50
Experiment: 7/50
Experiment: 8/50
Experiment: 9/50
Experiment: 10/50
Experiment: 11/50
Experiment: 12/50
Experiment: 13/50
Experiment: 14/50
Experiment: 15/50
Experiment: 16/50
Experiment: 17/50
Experiment: 18/50
Experiment: 19/50
Experiment: 20/50
Experiment: 21/50
Experiment: 22/50
Experiment: 23/50
Experiment: 24/50
Experiment: 25/50
Experiment: 26/50
Experiment: 27/50
Experiment: 28/50
Experiment: 29/50
Experiment: 30/50
Experiment: 31/50
Experiment: 32/50
Experiment: 33/50
Experiment: 34/50
Experiment: 35/50
Experiment: 36/50
Experiment: 37/50
Experiment: 38/50
Experiment: 39/50
Experiment: 40/50
Experiment: 41/50
Experiment: 42/50
Experiment: 43/50
Experiment: 44/50
Experiment: 45/50
Experiment: 46/50
Experiment: 47/50
Experiment: 48/50
Experiment: 49/50
Experiment: 50/50

        ID3_mean   ID3_std  lower_bound  upper_bound
0      39.491711  0.481737    38.046497    40.936924
1 

# Results

## Plot prediction

In [23]:
import plotly.express as px
import plotly.graph_objects as go

fig1 = go.Scatter(      x = frame_test.index,
                        y = frame_test['y']['ID3'][0],
                        name = 'Actual',
                        # color = hex_maroon
                        # title = "Log of Appliance Energy Consumption in Wh vs Time"
                    )

fig2 = go.Scatter(      x = frame_test.index,
                        y = frame_test['Prediction']['ID3'][0],
                        name = 'Predicted',
                        # color = hex_gold
                        # title = "Log of Appliance Energy Consumption in Wh vs Time"
                    )

data = [fig1, fig2]

fig = go.Figure(data = data)

fig.update_layout(      title = 'Forecast of test set',
                        xaxis_title = 'Timestamp',
                        yaxis_title = 'ID3 (€)')

fig.show()

In [25]:
import plotly.graph_objects as go

upper_trace = go.Scatter(
    y=test_uncertainty_df['upper_bound'],
    mode='lines',
    fill=None,
    name='99% Upper Confidence Bound'
    )

lower_trace = go.Scatter(
    y=test_uncertainty_df['lower_bound'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255, 211, 0, 0.5)',
    name='99% Lower Confidence Bound'
    )

real_trace = go.Scatter(
    y = frame_test['y']['ID3'][0],
    mode='lines',
    fill=None,
    name='Real Values'
    )

mean_trace = go.Scatter(
    y=test_uncertainty_df['ID3_mean'],
    mode='lines',
    fill=None,
    name='Mean Values'
    )

data = [upper_trace, lower_trace, mean_trace, real_trace]

fig = go.Figure(data=data)
fig.update_layout(title='Uncertainty Quantification for ID3 Test Data',
                   xaxis_title='Timestamp',
                   yaxis_title='ID3 (€)')

fig.layout.font.family = 'SF Mono'

fig.show()

## Metrics

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

print(smape(frame_test['y']['ID3'][0], frame_test['Prediction']['ID3'][0]))
print(smape(frame_test['y']['ID3'][1], frame_test['Prediction']['ID3'][1]))
print(smape(frame_test['y']['ID3'][2], frame_test['Prediction']['ID3'][2]))